# Fixture Difficulty Rating Rotation Optimization (2022-23 Season)

In FPL, Fixture Difficulty Rating (FDR) is a well-known metric. Simply put, FDR shows how difficult a team's game in every gameweek.

Finding pairs of teams that we can rotate is a common approach to simplify the FPL as a problem. Here, I will show how an FDR model can be written to find pairs, triplets, even under special conditions (e.g. FreeHit)

In [1]:
import pandas as pd
import sasoptpy as so
import requests
import json
import os
import random
from subprocess import Popen
from IPython.display import display, HTML
from math import exp
import string

In [2]:
def get_random_id(n):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(n))

In [3]:
ratings = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_global_rankings.csv")
ratings.head()

,rank,prev_rank,name,league,off,def,spi
0,1,1,Manchester City,Barclays Premier League,3.03,0.27,93.45
1,2,2,Liverpool,Barclays Premier League,2.90,0.24,93.26
2,3,3,Bayern Munich,German Bundesliga,3.41,0.59,91.63
3,4,4,Chelsea,Barclays Premier League,2.45,0.33,88.35
4,5,5,Paris Saint-Germain,French Ligue 1,2.79,0.65,85.82


In [4]:
hfa = 0.0
fixture = pd.read_excel("../data/ben_2022_23.xlsx", sheet_name="HA Schedule", header=2, index_col=0, usecols=range(2, 42), engine='openpyxl').drop(columns=['Unnamed: 3'])
fixture.index.name ='team'
fixture_original = fixture.copy()
# fixture = fixture.applymap(lambda x: x.upper())
# fixture.index = fixture.index.str.upper()
fix_dict = fixture.to_dict('index')
fixture.head()

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0
team,,,,,,,,,,,,,,,,,,,,,
ARS,cry,LEI,bou,FUL,AVL,mun,EVE,bre,TOT,LIV,...,LEE,liv,whu,SOU,mci,CHE,new,BHA,nfo,WOL
AVL,bou,EVE,cry,WHU,ars,MCI,lei,SOU,lee,nfo,...,che,NFO,NEW,bre,FUL,mun,wol,TOT,liv,BHA
BOU,AVL,mci,ARS,liv,WOL,nfo,BHA,new,BRE,LEI,...,FUL,lei,tot,WHU,sou,LEE,CHE,cry,MUN,eve
BRE,lei,MUN,ful,EVE,cry,LEE,sou,ARS,bou,new,...,bha,NEW,wol,AVL,che,NFO,liv,WHU,tot,MCI
BHA,mun,NEW,whu,LEE,ful,LEI,bou,CRY,liv,TOT,...,BRE,tot,che,MCI,nfo,WOL,EVE,ars,SOU,avl


In [5]:
teams = {
    'ARS': {'name': 'Arsenal'},
    'AVL':  {'name': 'Aston Villa'},
    'BOU': {'name': 'AFC Bournemouth'},
    'BRE':  {'name': 'Brentford'},
    'BHA':  {'name': 'Brighton and Hove Albion'},
    'CHE':  {'name': 'Chelsea'},
    'CRY':  {'name': 'Crystal Palace'},
    'EVE':  {'name': 'Everton'},
    'FUL': {'name': 'Fulham'},
    'LEI':  {'name': 'Leicester City'},
    'LEE': {'name':  'Leeds United'},
    'LIV': {'name':  'Liverpool'},
    'MCI': {'name':  'Manchester City'},
    'MUN': {'name':  'Manchester United'},
    'NEW': {'name':  'Newcastle'},
    'NFO': {'name': 'Nottingham Forest'},
    'SOU': {'name':  'Southampton'},
    'TOT': {'name':  'Tottenham Hotspur'},
    'WHU': {'name':  'West Ham United'},
    'WOL': {'name':  'Wolverhampton'}
}

In [6]:
for team, val in teams.items():
    # rating = ratings.loc[ratings.name == val['name'], target].values[0]
    # val['rating'] = rating
    val['off_rating'] = ratings.loc[ratings.name == val['name'], 'off'].values[0]
    val['def_rating'] = ratings.loc[ratings.name == val['name'], 'def'].values[0]
    val['spi_rating'] = ratings.loc[ratings.name == val['name'], 'spi'].values[0]

goal_target_rate = 1.38
avg_off = sum(v['off_rating'] for v in teams.values()) / 20
avg_def = sum(v['def_rating'] for v in teams.values()) / 20
off_adj = goal_target_rate / avg_off
def_adj = goal_target_rate / avg_def
for team, val in teams.items():
    val['off_adj'] = off_adj * val['off_rating']
    val['def_adj'] = def_adj * val['def_rating']

teams['LIV']

{'name': 'Liverpool',
 'off_rating': 2.9,
 'def_rating': 0.24,
 'spi_rating': 93.26,
 'off_adj': 1.956968215158924,
 'def_adj': 0.45809128630705387}

In [7]:
for team, val in teams.items():
    print(f"{team:.3s} {val['spi_rating']:.1f}")

tdf = pd.DataFrame([{'team': t, **v} for (t,v) in teams.items()])
tdf.sort_values(by='spi_rating')

ARS 80.2
AVL 76.2
BOU 58.9
BRE 72.4
BHA 80.0
CHE 88.3
CRY 76.5
EVE 63.7
FUL 64.1
LEI 71.3
LEE 63.0
LIV 93.3
MCI 93.5
MUN 74.7
NEW 71.7
NFO 56.5
SOU 61.9
TOT 84.1
WHU 73.0
WOL 69.6


,team,name,off_rating,def_rating,spi_rating,off_adj,def_adj
15,NFO,Nottingham Forest,1.45,0.99,56.51,0.978484,1.889627
2,BOU,AFC Bournemouth,1.56,1.00,58.93,1.052714,1.908714
16,SOU,Southampton,1.82,1.10,61.93,1.228166,2.099585
10,LEE,Leeds United,1.84,1.07,63.04,1.241663,2.042324
7,EVE,Everton,1.77,0.99,63.69,1.194425,1.889627
8,FUL,Fulham,1.81,1.00,64.11,1.221418,1.908714
19,WOL,Wolverhampton,1.78,0.76,69.58,1.201174,1.450622
9,LEI,Leicester City,1.98,0.83,71.31,1.336137,1.584232
14,NEW,Newcastle,1.84,0.71,71.71,1.241663,1.355187
3,BRE,Brentford,1.99,0.79,72.40,1.342885,1.507884


In [8]:
tdf['def_rating'].sum()

14.46

In [9]:
# Sets
team_list = list(teams.keys())
gameweeks = list(range(1,39))

In [10]:
def get_fdr_with_hfa(hfa=0, rating_type='rating', include_own_str=False):
    fdr = {}
    for t in team_list:
        for w in range(1,39):
            opp = fix_dict[t][w]
            if opp.islower(): # away
                if rating_type == 'def_rating':
                    fdr[t,w] = teams[opp.upper()][rating_type] / exp(hfa)
                else:
                    fdr[t,w] = teams[opp.upper()][rating_type] * exp(hfa)
            else: # home
                if rating_type == 'def_rating':
                    fdr[t,w] = teams[opp][rating_type] * exp(hfa) # opp concedes more goals
                else:
                    fdr[t,w] = teams[opp][rating_type] / exp(hfa)
    return fdr

In [11]:
pd.set_option('display.max_columns', None) 

In [12]:
def read_solution(m, sol_file="fdr.sol", solver='cbc'):
    with open(sol_file, 'r') as f:
        for v in m.get_variables():
            v.set_value(0)
        if solver == 'cbc':
            for line in f:
                if 'objective value' in line:
                    continue
                words = line.split()
                v = m.get_variable(words[1])
                v.set_value(float(words[2]))
        elif solver == 'highs':
            cols_started = False
            for line in f:
                if not cols_started and "# Columns" not in line:
                    continue
                elif "# Columns" in line:
                    cols_started = True
                    continue
                elif cols_started and line[0] != "#":
                    words = line.split()
                    v = m.get_variable(words[0])
                    try:
                        v.set_value(float(words[1]))
                    except:
                        print("Error", words[0], line)
                else:
                    return

                

def print_solution(m, gws, fdr):
    pick_team = m.get_variable('pick_team')
    pick_team_gw = m.get_variable('pick_team_gw')
    # Print solution
    selected_teams = []
    gameweek_picks = []
    for t in team_list:
        entry = {'team': t}
        if pick_team[t].get_value() > 0:
            selected_teams.append(t)
            for g in gws:
                entry.update({g: round(pick_team_gw[t,g].get_value() * fdr[t,g], 3) })
            gameweek_picks.append(entry)
    
    # Print and first table - values
    print(f'\nSelected: {" and ".join(selected_teams)}. Total FDR: {round(m.get_objective_value(),3)}, FDR per Fixture: {round(m.get_objective_value()/len(gws),2)}')
    pick_df = pd.DataFrame(gameweek_picks)
    s = pick_df.style
    colored_vals = lambda x: 'background-color: lightblue; color: black' if type(x) == float and x > 0 else 'color: white'
    s.applymap(colored_vals)
    # display(HTML(s.render().replace("000", "")))
    display(HTML(s.to_html(doctype_html=True).replace("000", "")))
    
    # Second table - names
    fr = fixture_original.reset_index()
    selected_fixture = fr[fr['team'].isin(selected_teams)].copy().reset_index(drop=True)
    selected_fixture = selected_fixture[['team'] + gws]
    s2 = selected_fixture.style
    def color_based_on_selection(cell):
        d = cell.copy()
        for c in d.columns:
            for r in d.index:
                if c == 'team':
                    d.loc[r,c] = ''
                elif pick_df.loc[r, c]:
                    d.loc[r, c] = 'background-color: green; color: white'
                else:
                    d.loc[r, c] = ''
        return d
    s2.apply(color_based_on_selection, axis=None)
    # display(HTML(s2.render().replace("000", "")))
    display(HTML(s2.to_html(doctype_html=True).replace("000", "")))
    return selected_teams

In [13]:
def solve_N_pair_problem(N=2, max_iter=1, first_gw=1, last_gw=38, among=[], exclude=[], hfa=0.0, rating_type='spi_rating'):
    fdr = get_fdr_with_hfa(hfa, rating_type)
    m = so.Model(name='N_rotation_pairs')
    team_list = list(teams.keys())
    gameweeks = list(range(first_gw, last_gw+1))
    pick_team = m.add_variables(team_list, vartype=so.binary, name='pick_team')
    pick_team_gw = m.add_variables(team_list, gameweeks, vartype=so.binary, name='pick_team_gw')

    if len(exclude) > 0:
        m.add_constraints((pick_team[t] == 0 for t in exclude), name='disable_team')
    if len(among) > 0:
        m.add_constraints((pick_team[t] == 0 for t in team_list if t not in among), name='disable_team')

    m.add_constraint(so.expr_sum(pick_team[t] for t in team_list) == N, name='pick_2')
    m.add_constraints((so.expr_sum(pick_team_gw[t, g] for t in team_list) == 1 for g in gameweeks), name='pick_1_per_gw')
    m.add_constraints((pick_team_gw[t,g] <= pick_team[t] for t in team_list for g in gameweeks), name='valid_picks_only')

    # Force using each team at least once
    m.add_constraints((so.expr_sum(pick_team_gw[t,g] for g in gameweeks) >= pick_team[t] for t in team_list), name='force_use')

    obj_mult = 1 if rating_type != 'def_rating' else -1
    m.set_objective(obj_mult * so.expr_sum(fdr[t, g] * pick_team_gw[t, g] for t in team_list for g in gameweeks), sense='N', name='total_fdr')

    m.export_mps("fdr.mps")
    # command = "cbc fdr.mps solve solu fdr.sol"
    command = "highs --model_file fdr.mps --solution_file fdr.sol"
    Popen(command).wait()
    read_solution(m, sol_file="fdr.sol", solver='highs')
    selected_teams = print_solution(m, gameweeks, fdr)
    for it in range(1, max_iter):
        c = m.add_constraint(so.expr_sum(pick_team[t] for t in selected_teams) <= N-1, name=f'cutoff_{it}')
        m.export_mps("fdr.mps")
        Popen(command).wait()
        read_solution(m, sol_file="fdr.sol", solver="highs")
        selected_teams = print_solution(m, gameweeks, fdr)

In [14]:
teams.keys()

dict_keys(['ARS', 'AVL', 'BOU', 'BRE', 'BHA', 'CHE', 'CRY', 'EVE', 'FUL', 'LEI', 'LEE', 'LIV', 'MCI', 'MUN', 'NEW', 'NFO', 'SOU', 'TOT', 'WHU', 'WOL'])

In [15]:
solve_N_pair_problem(N=1, max_iter=3, last_gw=8, hfa=0.0, rating_type='spi_rating')

NOTE: Initialized model N_rotation_pairs.

Selected: BHA. Total FDR: 553.27, FDR per Fixture: 69.16


,team,1,2,3,4,5,6,7,8
0,BHA,74.670,71.710,73.020,63.040,64.110,71.310,58.930,76.480


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,BHA,mun,NEW,whu,LEE,ful,LEI,bou,CRY



Selected: BRE. Total FDR: 555.41, FDR per Fixture: 69.43


,team,1,2,3,4,5,6,7,8
0,BRE,71.310,74.670,64.110,63.690,76.480,63.040,61.930,80.180


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,BRE,lei,MUN,ful,EVE,cry,LEE,sou,ARS



Selected: ARS. Total FDR: 557.81, FDR per Fixture: 69.73


,team,1,2,3,4,5,6,7,8
0,ARS,76.480,71.310,58.930,64.110,76.220,74.670,63.690,72.400


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,ARS,cry,LEI,bou,FUL,AVL,mun,EVE,bre


In [16]:
solve_N_pair_problem(N=2, max_iter=3, last_gw=8, exclude=['LIV', 'MCI', 'CHE', 'ARS', 'TOT'], hfa=0.0, rating_type='spi_rating')

NOTE: Initialized model N_rotation_pairs.

Selected: NFO and WOL. Total FDR: 507.56, FDR per Fixture: 63.45


,team,1,2,3,4,5,6,7,8
0,NFO,0.,0.,63.690,0.,0.,58.930,63.040,64.110
1,WOL,63.040,64.110,0.,71.710,58.930,0.,0.,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,NFO,new,WHU,eve,TOT,mci,BOU,lee,FUL
1,WOL,lee,FUL,tot,NEW,bou,SOU,liv,MCI



Selected: AVL and BRE. Total FDR: 513.8, FDR per Fixture: 64.23


,team,1,2,3,4,5,6,7,8
0,AVL,58.930,63.690,0.,0.,0.,0.,0.,61.930
1,BRE,0.,0.,64.110,63.690,76.480,63.040,61.930,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,AVL,bou,EVE,cry,WHU,ars,MCI,lei,SOU
1,BRE,lei,MUN,ful,EVE,cry,LEE,sou,ARS



Selected: AVL and BHA. Total FDR: 514.96, FDR per Fixture: 64.37


,team,1,2,3,4,5,6,7,8
0,AVL,58.930,63.690,0.,0.,0.,0.,0.,61.930
1,BHA,0.,0.,73.020,63.040,64.110,71.310,58.930,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,AVL,bou,EVE,cry,WHU,ars,MCI,lei,SOU
1,BHA,mun,NEW,whu,LEE,ful,LEI,bou,CRY


In [17]:
solve_N_pair_problem(N=2, max_iter=3, last_gw=8, exclude=['LIV', 'MCI', 'CHE'], hfa=0.0, rating_type='off_rating')

NOTE: Initialized model N_rotation_pairs.

Selected: LEE and TOT. Total FDR: 13.84, FDR per Fixture: 1.73


,team,1,2,3,4,5,6,7,8
0,LEE,1.780,1.820,0.,0.,1.770,0.,1.450,0.
1,TOT,0.,0.,1.780,1.450,0.,1.810,0.,1.980


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,LEE,WOL,sou,CHE,bha,EVE,bre,NFO,mun
1,TOT,SOU,che,WOL,nfo,whu,FUL,mci,LEI



Selected: BHA and NEW. Total FDR: 13.86, FDR per Fixture: 1.73


,team,1,2,3,4,5,6,7,8
0,BHA,0.,1.840,1.980,0.,1.810,0.,1.560,0.
1,NEW,1.450,0.,0.,1.780,0.,1.880,0.,1.560


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,BHA,mun,NEW,whu,LEE,ful,LEI,bou,CRY
1,NEW,NFO,bha,MCI,wol,liv,CRY,whu,BOU



Selected: TOT and WHU. Total FDR: 13.9, FDR per Fixture: 1.74


,team,1,2,3,4,5,6,7,8
0,TOT,1.820,0.,1.780,1.450,1.980,1.810,0.,0.
1,WHU,0.,1.450,0.,0.,0.,0.,1.840,1.770


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,TOT,SOU,che,WOL,nfo,whu,FUL,mci,LEI
1,WHU,MCI,nfo,BHA,avl,TOT,che,NEW,eve


In [18]:
solve_N_pair_problem(N=2, max_iter=3, last_gw=8, exclude=['LIV', 'MCI', 'CHE'], hfa=0.0, rating_type='def_rating')

NOTE: Initialized model N_rotation_pairs.

Selected: ARS and WOL. Total FDR: -7.95, FDR per Fixture: -0.99


,team,1,2,3,4,5,6,7,8
0,ARS,0.,0.,1.,1.,0.,0.,0.990,0.790
1,WOL,1.070,1.,0.,0.,1.,1.100,0.,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,ARS,cry,LEI,bou,FUL,AVL,mun,EVE,bre
1,WOL,lee,FUL,tot,NEW,bou,SOU,liv,MCI



Selected: NFO and WOL. Total FDR: -7.94, FDR per Fixture: -0.99


,team,1,2,3,4,5,6,7,8
0,NFO,0.,0.,0.990,0.,0.,0.,1.070,1.
1,WOL,1.070,1.,0.,0.710,1.,1.100,0.,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,NFO,new,WHU,eve,TOT,mci,BOU,lee,FUL
1,WOL,lee,FUL,tot,NEW,bou,SOU,liv,MCI



Selected: BRE and WOL. Total FDR: -7.85, FDR per Fixture: -0.98


,team,1,2,3,4,5,6,7,8
0,BRE,0.,0.,1.,0.990,0.,0.,1.100,0.590
1,WOL,1.070,1.,0.,0.,1.,1.100,0.,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,BRE,lei,MUN,ful,EVE,cry,LEE,sou,ARS
1,WOL,lee,FUL,tot,NEW,bou,SOU,liv,MCI


In [19]:
solve_N_pair_problem(N=2, max_iter=1, rating_type='off_rating')

NOTE: Initialized model N_rotation_pairs.

Selected: ARS and NEW. Total FDR: 67.88, FDR per Fixture: 1.79


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,ARS,0.,1.980,1.560,0.,2.020,0.,1.770,0.,0.,0.,1.840,0.,1.820,1.450,0.,1.780,0.,0.,1.840,0.,0.,1.770,0.,2.020,1.980,1.560,0.,0.,1.840,0.,1.980,1.820,0.,0.,1.840,0.,1.450,1.780
1,NEW,1.450,0.,0.,1.780,0.,1.880,0.,1.560,1.810,1.990,0.,1.770,0.,0.,1.820,0.,1.980,1.840,0.,1.810,1.880,0.,1.560,0.,0.,0.,1.780,1.450,0.,1.990,0.,0.,1.770,1.820,0.,1.840,0.,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.,9.,10.,11.,12.,13.,14.,15.,16.,17.,18.,19.,20.,21.,22.,23.,24.,25.,26.,27.,28.,29.,30.,31.,32.,33.,34.,35.,36.,37.,38.
0,ARS,cry,LEI,bou,FUL,AVL,mun,EVE,bre,TOT,LIV,lee,MCI,sou,NFO,che,wol,WHU,bha,NEW,tot,MUN,eve,BRE,avl,lei,BOU,ful,CRY,LEE,liv,whu,SOU,mci,CHE,new,BHA,nfo,WOL
1,NEW,NFO,bha,MCI,wol,liv,CRY,whu,BOU,ful,BRE,mun,EVE,tot,AVL,sou,CHE,lei,LEE,ars,FUL,cry,WHU,bou,LIV,BHA,mci,WOL,nfo,MUN,bre,avl,TOT,eve,SOU,ARS,lee,LEI,che


In [20]:
solve_N_pair_problem(N=2, max_iter=1, rating_type='spi_rating')

NOTE: Initialized model N_rotation_pairs.

Selected: ARS and CHE and NEW and TOT. Total FDR: 2488.95, FDR per Fixture: 65.5


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,ARS,0.,71.310,58.930,64.110,76.220,74.670,63.690,0.,0.,0.,63.040,0.,61.930,56.510,0.,69.580,-0.,0.,71.710,0.,74.670,63.690,-0.,76.220,71.310,58.930,64.110,0.,63.040,0.,73.020,61.930,0.,0.,71.710,-0.,56.510,69.580
1,CHE,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,-0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.
2,NEW,56.510,0.,0.,0.,0.,0.,-0.,58.930,64.110,72.400,-0.,63.690,0.,0.,61.930,0.,71.310,63.040,0.,64.110,0.,0.,58.930,0.,0.,0.,-0.,56.510,-0.,72.400,0.,0.,63.690,61.930,0.,63.040,-0.,0.
3,TOT,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.,9.,10.,11.,12.,13.,14.,15.,16.,17.,18.,19.,20.,21.,22.,23.,24.,25.,26.,27.,28.,29.,30.,31.,32.,33.,34.,35.,36.,37.,38.
0,ARS,cry,LEI,bou,FUL,AVL,mun,EVE,bre,TOT,LIV,lee,MCI,sou,NFO,che,wol,WHU,bha,NEW,tot,MUN,eve,BRE,avl,lei,BOU,ful,CRY,LEE,liv,whu,SOU,mci,CHE,new,BHA,nfo,WOL
1,CHE,eve,TOT,lee,LEI,sou,WHU,ful,LIV,cry,WOL,avl,bre,MUN,bha,ARS,new,BOU,nfo,MCI,CRY,liv,FUL,whu,SOU,tot,LEE,lei,EVE,AVL,wol,BHA,mun,BRE,ars,bou,NFO,mci,NEW
2,NEW,NFO,bha,MCI,wol,liv,CRY,whu,BOU,ful,BRE,mun,EVE,tot,AVL,sou,CHE,lei,LEE,ars,FUL,cry,WHU,bou,LIV,BHA,mci,WOL,nfo,MUN,bre,avl,TOT,eve,SOU,ARS,lee,LEI,che
3,TOT,SOU,che,WOL,nfo,whu,FUL,mci,LEI,ars,bha,EVE,mun,NEW,bou,LIV,LEE,bre,AVL,cry,ARS,ful,MCI,lei,WHU,CHE,wol,NFO,sou,eve,BHA,BOU,new,MUN,liv,CRY,avl,BRE,lee


In [21]:
solve_N_pair_problem(N=3, max_iter=1, rating_type='spi_rating')

NOTE: Initialized model N_rotation_pairs.

Selected: ARS and BRE and LIV and MCI and NEW and TOT and WHU. Total FDR: 2380.09, FDR per Fixture: 62.63


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,ARS,0.,0.,58.930,64.110,0.,74.670,63.690,0.,0.,0.,63.040,0.,61.930,56.510,0.,69.580,0.,0.,71.710,0.,0.,63.690,0.,0.,0.,58.930,64.110,0.,63.040,0.,0.,61.930,0.,0.,0.,0.,56.510,69.580
1,BRE,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.
2,LIV,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.
3,MCI,0.,58.930,0.,0.,56.510,0.,0.,0.,0.,61.930,0.,0.,0.,0.,0.,0.,63.040,0.,0.,0.,69.580,0.,0.,56.510,58.930,0.,0.,0.,0.,61.930,71.310,0.,0.,0.,63.040,0.,0.,0.
4,NEW,56.510,0.,0.,0.,0.,0.,0.,58.930,64.110,0.,0.,63.690,0.,0.,61.930,0.,0.,63.040,0.,64.110,0.,0.,58.930,0.,0.,0.,0.,56.510,0.,0.,0.,0.,63.690,61.930,0.,63.040,0.,0.
5,TOT,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.
6,WHU,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.,9.,10.,11.,12.,13.,14.,15.,16.,17.,18.,19.,20.,21.,22.,23.,24.,25.,26.,27.,28.,29.,30.,31.,32.,33.,34.,35.,36.,37.,38.
0,ARS,cry,LEI,bou,FUL,AVL,mun,EVE,bre,TOT,LIV,lee,MCI,sou,NFO,che,wol,WHU,bha,NEW,tot,MUN,eve,BRE,avl,lei,BOU,ful,CRY,LEE,liv,whu,SOU,mci,CHE,new,BHA,nfo,WOL
1,BRE,lei,MUN,ful,EVE,cry,LEE,sou,ARS,bou,new,BHA,CHE,avl,WOL,nfo,mci,TOT,whu,LIV,BOU,lee,SOU,ars,CRY,mun,FUL,eve,LEI,bha,NEW,wol,AVL,che,NFO,liv,WHU,tot,MCI
2,LIV,ful,CRY,mun,BOU,NEW,eve,WOL,che,BHA,ars,MCI,WHU,nfo,LEE,tot,SOU,avl,LEI,bre,bha,CHE,wol,EVE,new,cry,MUN,bou,FUL,mci,ARS,lee,NFO,whu,TOT,BRE,lei,AVL,sou
3,MCI,whu,BOU,new,CRY,NFO,avl,TOT,wol,MUN,SOU,liv,ars,BHA,lei,FUL,BRE,lee,EVE,che,mun,WOL,tot,AVL,nfo,bou,NEW,cry,WHU,LIV,sou,LEI,bha,ARS,ful,LEE,eve,CHE,bre
4,NEW,NFO,bha,MCI,wol,liv,CRY,whu,BOU,ful,BRE,mun,EVE,tot,AVL,sou,CHE,lei,LEE,ars,FUL,cry,WHU,bou,LIV,BHA,mci,WOL,nfo,MUN,bre,avl,TOT,eve,SOU,ARS,lee,LEI,che
5,TOT,SOU,che,WOL,nfo,whu,FUL,mci,LEI,ars,bha,EVE,mun,NEW,bou,LIV,LEE,bre,AVL,cry,ARS,ful,MCI,lei,WHU,CHE,wol,NFO,sou,eve,BHA,BOU,new,MUN,liv,CRY,avl,BRE,lee
6,WHU,MCI,nfo,BHA,avl,TOT,che,NEW,eve,WOL,FUL,sou,liv,BOU,mun,CRY,LEI,ars,BRE,lee,wol,EVE,new,CHE,tot,NFO,bha,AVL,mci,SOU,ful,ARS,bou,LIV,cry,MUN,bre,LEE,lei


In [22]:
# Free hit version
def solve_N_pair_problem_fh(N=2, max_iter=1, hfa=0.0, rating_type='spi_rating'):
    fdr = get_fdr_with_hfa(hfa, rating_type)
    m = so.Model(name='N_rotation_pairs')
    team_list = list(teams.keys())
    gameweeks = list(range(1,39))
    pick_team = m.add_variables(team_list, vartype=so.binary, name='pick_team')
    pick_team_gw = m.add_variables(team_list, gameweeks, vartype=so.binary, name='pick_team_gw')
    pick_free_hit = m.add_variables(team_list, gameweeks, vartype=so.binary, name='pick_free_hit')

    m.add_constraint(so.expr_sum(pick_team[t] for t in team_list) == N, name='pick_2')
    m.add_constraints((so.expr_sum(pick_team_gw[t, g] + pick_free_hit[t, g] for t in team_list) == 1 for g in gameweeks), name='pick_1_per_gw')
    m.add_constraints((pick_team_gw[t,g] <= pick_team[t] for t in team_list for g in gameweeks), name='valid_picks_only')
    m.add_constraint((so.expr_sum(pick_free_hit[t, g] for t in team_list for g in gameweeks) <= 1), name='single_free_hit')

    # Force using each team at least once
    m.add_constraints((so.expr_sum(pick_team_gw[t,g] for g in gameweeks) >= pick_team[t] for t in team_list), name='force_use')

    obj_mult = 1 if rating_type != 'def_rating' else -1
    m.set_objective(obj_mult * so.expr_sum(fdr[t, g] * (pick_team_gw[t, g] + pick_free_hit[t,g]) for t in team_list for g in gameweeks), sense='N', name='total_fdr')

    m.export_mps("fdr.mps")
    # command = "cbc fdr.mps solve solu fdr.sol"
    command = "highs --model_file fdr.mps --solution_file fdr.sol"
    Popen(command).wait()
    read_solution(m, "fdr.sol", "highs")

    def print_free_hit_sol():
        selected_teams = []
        gameweek_picks = []
        for t in team_list:
            entry = {'team': t}
            if pick_team[t].get_value() > 0 or so.expr_sum(pick_free_hit[t,g] for g in gameweeks).get_value() > 0:
                selected_teams.append(t)
                for g in gameweeks:
                    entry.update({g: round((pick_team_gw[t,g] + pick_free_hit[t,g]).get_value() * fdr[t,g], 3) })
                gameweek_picks.append(entry)
        print(f'Selected: {" and ".join(selected_teams)}. Total FDR: {round(m.get_objective_value(),3)}')
        pick_df = pd.DataFrame(gameweek_picks)
        s = pick_df.style
        colored_vals = lambda x: 'background-color: lightblue; color: black' if type(x) == float and x > 0 else 'color: black'
        s.applymap(colored_vals)
        display(HTML(s.to_html(doctype_html=True).replace("000", "")))

        # Second table - names
        fr = fixture_original.reset_index()
        selected_fixture = fr[fr['team'].isin(selected_teams)].copy().reset_index(drop=True)
        s2 = selected_fixture.style
        def color_based_on_selection(cell):
            d = cell.copy()
            for c in d.columns:
                for r in d.index:
                    if c == 'team':
                        d.loc[r, c] = ''
                    elif pick_df.loc[r, c]:
                        d.loc[r, c] = 'background-color: green; color: white'
                    else:
                        d.loc[r, c] = ''
            return d
        s2.apply(color_based_on_selection, axis=None)
        display(HTML(s2.to_html(doctype_html=True)))

        return selected_teams

    selected_teams = print_free_hit_sol()
    for it in range(1, max_iter):
        c = m.add_constraint(so.expr_sum(pick_team[t] for t in selected_teams) <= N-1, name=f'cutoff_{it}')
        m.export_mps("fdr.mps")
        Popen(command).wait()
        read_solution(m, "fdr.sol", "highs")
        selected_teams = print_free_hit_sol()

In [23]:
# solve_N_pair_problem_fh(2, 3, rating_type='spi_rating')

In [24]:
solve_N_pair_problem(N=2, max_iter=1, first_gw=1, last_gw=10)

NOTE: Initialized model N_rotation_pairs.

Selected: AVL and BRE. Total FDR: 629.24, FDR per Fixture: 62.92


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,58.930,63.690,0.,0.,0.,0.,0.,61.930,0.,56.510
1,BRE,0.,0.,64.110,63.690,76.480,63.040,61.930,0.,58.930,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.,9.,10.
0,AVL,bou,EVE,cry,WHU,ars,MCI,lei,SOU,lee,nfo
1,BRE,lei,MUN,ful,EVE,cry,LEE,sou,ARS,bou,new


In [25]:
def solve_N_pick_K_pair_problem(N=3, K=2, max_iter=1, first_gw=1, last_gw=38, exclude=[], hfa=0.0, rating_type='spi_rating'):
    if last_gw > 38:
        return ["-"]
    fdr = get_fdr_with_hfa(hfa, rating_type)
    problem_name = get_random_id(10)
    m = so.Model(name=f'N_K_rotation_problem_name')
    team_list = list(teams.keys())
    gameweeks = list(range(first_gw, last_gw+1))
    pick_team = m.add_variables(team_list, vartype=so.binary, name='pick_team')
    pick_team_gw = m.add_variables(team_list, gameweeks, vartype=so.binary, name='pick_team_gw')

    if len(exclude) > 0:
        m.add_constraints((pick_team[t] == 0 for t in exclude), name='disable_teams')

    m.add_constraint(so.expr_sum(pick_team[t] for t in team_list) == N, name='pick_2')
    m.add_constraints((so.expr_sum(pick_team_gw[t, g] for t in team_list) == K for g in gameweeks), name='pick_1_per_gw')
    m.add_constraints((pick_team_gw[t,g] <= pick_team[t] for t in team_list for g in gameweeks), name='valid_picks_only')

    # Force using each team at least once
    m.add_constraints((so.expr_sum(pick_team_gw[t,g] for g in gameweeks) >= pick_team[t] for t in team_list), name='force_use')

    obj_mult = 1 if rating_type != 'def_rating' else -1
    m.set_objective(obj_mult * so.expr_sum(fdr[t, g] * pick_team_gw[t, g] for t in team_list for g in gameweeks), sense='N', name='total_fdr')

    m.export_mps(f"tmp/{problem_name}.mps")
    command = f"highs --model_file tmp/{problem_name}.mps --solution_file tmp/{problem_name}.sol"
    # command = f"cbc tmp/{problem_name}.mps solve solu tmp/{problem_name}.sol"
    Popen(command).wait()
    read_solution(m, f'tmp/{problem_name}.sol', 'highs')
    selected_teams = print_solution(m, gameweeks, fdr)
    for it in range(1, max_iter):
        c = m.add_constraint(so.expr_sum(pick_team[t] for t in selected_teams) <= N-1, name=f'cutoff_{it}')
        m.export_mps(f"tmp/{problem_name}.mps")
        Popen(command).wait()
        read_solution(m, f'tmp/{problem_name}.sol', 'highs')
        selected_teams = print_solution(m, gameweeks, fdr)

    return selected_teams

In [26]:
solve_N_pick_K_pair_problem(first_gw=1, last_gw=38, max_iter=1, rating_type='spi_rating')

NOTE: Initialized model N_K_rotation_problem_name.

Selected: BHA and CHE and WHU. Total FDR: 5140.39, FDR per Fixture: 135.27


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,BHA,74.670,71.710,73.020,63.040,64.110,71.310,58.930,76.480,0.,0.,72.400,56.510,0.,0.,69.580,0.,61.930,0.,63.690,0.,71.310,58.930,76.480,64.110,71.710,73.020,63.040,74.670,72.400,0.,0.,0.,56.510,69.580,63.690,0.,61.930,0.
1,CHE,63.690,0.,63.040,71.310,61.930,73.020,64.110,0.,76.480,69.580,0.,72.400,74.670,80.030,0.,71.710,58.930,56.510,0.,76.480,0.,64.110,73.020,61.930,0.,63.040,71.310,63.690,0.,69.580,80.030,74.670,72.400,0.,58.930,56.510,0.,71.710
2,WHU,0.,56.510,0.,0.,0.,0.,0.,63.690,69.580,64.110,61.930,0.,58.930,74.670,76.480,71.310,0.,72.400,63.040,69.580,63.690,0.,0.,0.,56.510,0.,0.,0.,61.930,64.110,80.180,58.930,0.,76.480,0.,72.400,63.040,71.310


,team,1.,2.,3.,4.,5.,6.,7.,8.,9.,10.,11.,12.,13.,14.,15.,16.,17.,18.,19.,20.,21.,22.,23.,24.,25.,26.,27.,28.,29.,30.,31.,32.,33.,34.,35.,36.,37.,38.
0,BHA,mun,NEW,whu,LEE,ful,LEI,bou,CRY,liv,TOT,bre,NFO,mci,CHE,wol,AVL,sou,ARS,eve,LIV,lei,BOU,cry,FUL,new,WHU,lee,MUN,BRE,tot,che,MCI,nfo,WOL,EVE,ars,SOU,avl
1,CHE,eve,TOT,lee,LEI,sou,WHU,ful,LIV,cry,WOL,avl,bre,MUN,bha,ARS,new,BOU,nfo,MCI,CRY,liv,FUL,whu,SOU,tot,LEE,lei,EVE,AVL,wol,BHA,mun,BRE,ars,bou,NFO,mci,NEW
2,WHU,MCI,nfo,BHA,avl,TOT,che,NEW,eve,WOL,FUL,sou,liv,BOU,mun,CRY,LEI,ars,BRE,lee,wol,EVE,new,CHE,tot,NFO,bha,AVL,mci,SOU,ful,ARS,bou,LIV,cry,MUN,bre,LEE,lei


['BHA', 'CHE', 'WHU']

In [27]:
solve_N_pick_K_pair_problem(first_gw=1, last_gw=10, max_iter=1, exclude=['MCI'])

NOTE: Initialized model N_K_rotation_problem_name.

Selected: AVL and BRE and TOT. Total FDR: 1306.43, FDR per Fixture: 130.64


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,58.930,63.690,0.,0.,0.,0.,71.310,61.930,63.040,56.510
1,BRE,0.,74.670,64.110,63.690,76.480,63.040,61.930,0.,58.930,71.710
2,TOT,61.930,0.,69.580,56.510,73.020,64.110,0.,71.310,0.,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.,9.,10.
0,AVL,bou,EVE,cry,WHU,ars,MCI,lei,SOU,lee,nfo
1,BRE,lei,MUN,ful,EVE,cry,LEE,sou,ARS,bou,new
2,TOT,SOU,che,WOL,nfo,whu,FUL,mci,LEI,ars,bha


['AVL', 'BRE', 'TOT']

In [28]:
solve_N_pick_K_pair_problem(N=4, K=3, first_gw=1, last_gw=38, max_iter=1)


NOTE: Initialized model N_K_rotation_problem_name.

Selected: ARS and LEI and MCI and NEW. Total FDR: 7837.5, FDR per Fixture: 206.25


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,ARS,0.,71.310,58.930,64.110,76.220,74.670,63.690,72.400,0.,0.,63.040,0.,61.930,56.510,0.,69.580,0.,80.030,71.710,0.,74.670,63.690,72.400,76.220,71.310,58.930,64.110,0.,63.040,0.,73.020,61.930,0.,0.,71.710,80.030,56.510,69.580
1,LEI,72.400,0.,61.930,0.,74.670,0.,76.220,0.,56.510,58.930,76.480,63.040,69.580,0.,63.690,73.020,71.710,0.,64.110,56.510,0.,76.220,0.,74.670,0.,61.930,0.,72.400,76.480,58.930,0.,69.580,63.040,63.690,64.110,0.,71.710,73.020
2,MCI,73.020,58.930,71.710,76.480,56.510,76.220,0.,69.580,74.670,61.930,0.,80.180,80.030,71.310,64.110,72.400,63.040,63.690,0.,74.670,69.580,0.,76.220,56.510,58.930,71.710,76.480,73.020,0.,61.930,71.310,80.030,80.180,64.110,63.040,63.690,0.,72.400
3,NEW,56.510,80.030,0.,69.580,0.,76.480,73.020,58.930,64.110,72.400,74.670,63.690,0.,76.220,61.930,0.,71.310,63.040,80.180,64.110,76.480,73.020,58.930,0.,80.030,0.,69.580,56.510,74.670,72.400,76.220,0.,63.690,61.930,0.,63.040,71.310,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.,9.,10.,11.,12.,13.,14.,15.,16.,17.,18.,19.,20.,21.,22.,23.,24.,25.,26.,27.,28.,29.,30.,31.,32.,33.,34.,35.,36.,37.,38.
0,ARS,cry,LEI,bou,FUL,AVL,mun,EVE,bre,TOT,LIV,lee,MCI,sou,NFO,che,wol,WHU,bha,NEW,tot,MUN,eve,BRE,avl,lei,BOU,ful,CRY,LEE,liv,whu,SOU,mci,CHE,new,BHA,nfo,WOL
1,LEI,BRE,ars,SOU,che,MUN,bha,AVL,tot,NFO,bou,CRY,LEE,wol,MCI,eve,whu,NEW,liv,FUL,nfo,BHA,avl,TOT,mun,ARS,sou,CHE,bre,cry,BOU,mci,WOL,lee,EVE,ful,LIV,new,WHU
2,MCI,whu,BOU,new,CRY,NFO,avl,TOT,wol,MUN,SOU,liv,ars,BHA,lei,FUL,BRE,lee,EVE,che,mun,WOL,tot,AVL,nfo,bou,NEW,cry,WHU,LIV,sou,LEI,bha,ARS,ful,LEE,eve,CHE,bre
3,NEW,NFO,bha,MCI,wol,liv,CRY,whu,BOU,ful,BRE,mun,EVE,tot,AVL,sou,CHE,lei,LEE,ars,FUL,cry,WHU,bou,LIV,BHA,mci,WOL,nfo,MUN,bre,avl,TOT,eve,SOU,ARS,lee,LEI,che


['ARS', 'LEI', 'MCI', 'NEW']

In [29]:
solve_N_pick_K_pair_problem(first_gw=1, last_gw=38, max_iter=1, exclude=['MCI', 'CHE', 'LIV', 'MUN'])

NOTE: Initialized model N_K_rotation_problem_name.

Selected: AVL and BHA and TOT. Total FDR: 5151.51, FDR per Fixture: 135.57


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,AVL,58.930,63.690,0.,0.,0.,0.,71.310,61.930,63.040,56.510,0.,64.110,72.400,71.710,74.670,0.,0.,0.,69.580,63.040,61.930,71.310,0.,0.,63.690,0.,0.,58.930,0.,56.510,71.710,72.400,64.110,74.670,69.580,0.,0.,0.
1,BHA,0.,71.710,73.020,63.040,64.110,71.310,58.930,0.,0.,0.,72.400,56.510,0.,0.,69.580,76.220,61.930,80.180,63.690,0.,0.,58.930,76.480,64.110,71.710,73.020,63.040,0.,72.400,0.,0.,0.,56.510,69.580,63.690,80.180,61.930,76.220
2,TOT,61.930,0.,69.580,56.510,73.020,64.110,0.,71.310,80.180,80.030,63.690,0.,71.710,58.930,0.,63.040,72.400,76.220,0.,80.180,64.110,0.,71.310,73.020,0.,69.580,56.510,61.930,63.690,80.030,58.930,71.710,0.,0.,0.,76.220,72.400,63.040


,team,1.,2.,3.,4.,5.,6.,7.,8.,9.,10.,11.,12.,13.,14.,15.,16.,17.,18.,19.,20.,21.,22.,23.,24.,25.,26.,27.,28.,29.,30.,31.,32.,33.,34.,35.,36.,37.,38.
0,AVL,bou,EVE,cry,WHU,ars,MCI,lei,SOU,lee,nfo,CHE,ful,BRE,new,MUN,bha,LIV,tot,WOL,LEE,sou,LEI,mci,ARS,eve,CRY,whu,BOU,che,NFO,NEW,bre,FUL,mun,wol,TOT,liv,BHA
1,BHA,mun,NEW,whu,LEE,ful,LEI,bou,CRY,liv,TOT,bre,NFO,mci,CHE,wol,AVL,sou,ARS,eve,LIV,lei,BOU,cry,FUL,new,WHU,lee,MUN,BRE,tot,che,MCI,nfo,WOL,EVE,ars,SOU,avl
2,TOT,SOU,che,WOL,nfo,whu,FUL,mci,LEI,ars,bha,EVE,mun,NEW,bou,LIV,LEE,bre,AVL,cry,ARS,ful,MCI,lei,WHU,CHE,wol,NFO,sou,eve,BHA,BOU,new,MUN,liv,CRY,avl,BRE,lee


['AVL', 'BHA', 'TOT']

In [30]:
solve_N_pick_K_pair_problem(first_gw=1, last_gw=10, max_iter=1, exclude=['MCI', 'CHE', 'LIV', 'MUN'])

NOTE: Initialized model N_K_rotation_problem_name.

Selected: AVL and BRE and TOT. Total FDR: 1306.43, FDR per Fixture: 130.64


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,58.930,63.690,0.,0.,0.,0.,71.310,61.930,63.040,56.510
1,BRE,0.,74.670,64.110,63.690,76.480,63.040,61.930,0.,58.930,71.710
2,TOT,61.930,0.,69.580,56.510,73.020,64.110,0.,71.310,0.,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.,9.,10.
0,AVL,bou,EVE,cry,WHU,ars,MCI,lei,SOU,lee,nfo
1,BRE,lei,MUN,ful,EVE,cry,LEE,sou,ARS,bou,new
2,TOT,SOU,che,WOL,nfo,whu,FUL,mci,LEI,ars,bha


['AVL', 'BRE', 'TOT']

In [31]:
solve_N_pick_K_pair_problem(N=3, K=2, first_gw=1, last_gw=38, max_iter=1, exclude=['MCI', 'CHE', 'LIV', 'MUN', 'LEI', 'ARS', 'TOT'])

NOTE: Initialized model N_K_rotation_problem_name.

Selected: BRE and BHA and FUL. Total FDR: 5153.2, FDR per Fixture: 135.61


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,BRE,71.310,0.,64.110,63.690,76.480,63.040,61.930,0.,58.930,71.710,0.,0.,76.220,69.580,56.510,0.,0.,73.020,0.,58.930,63.040,61.930,0.,76.480,0.,64.110,63.690,71.310,0.,71.710,69.580,76.220,0.,56.510,0.,73.020,0.,0.
1,BHA,74.670,71.710,0.,63.040,64.110,71.310,58.930,76.480,0.,0.,72.400,56.510,0.,0.,69.580,76.220,61.930,0.,63.690,0.,71.310,58.930,76.480,64.110,71.710,0.,63.040,74.670,72.400,0.,0.,0.,56.510,69.580,63.690,0.,61.930,76.220
2,FUL,0.,69.580,72.400,0.,0.,0.,0.,56.510,71.710,73.020,58.930,76.220,63.040,63.690,0.,74.670,76.480,61.930,71.310,71.710,0.,0.,56.510,0.,69.580,72.400,0.,0.,58.930,73.020,63.690,63.040,76.220,0.,71.310,61.930,76.480,74.670


,team,1.,2.,3.,4.,5.,6.,7.,8.,9.,10.,11.,12.,13.,14.,15.,16.,17.,18.,19.,20.,21.,22.,23.,24.,25.,26.,27.,28.,29.,30.,31.,32.,33.,34.,35.,36.,37.,38.
0,BRE,lei,MUN,ful,EVE,cry,LEE,sou,ARS,bou,new,BHA,CHE,avl,WOL,nfo,mci,TOT,whu,LIV,BOU,lee,SOU,ars,CRY,mun,FUL,eve,LEI,bha,NEW,wol,AVL,che,NFO,liv,WHU,tot,MCI
1,BHA,mun,NEW,whu,LEE,ful,LEI,bou,CRY,liv,TOT,bre,NFO,mci,CHE,wol,AVL,sou,ARS,eve,LIV,lei,BOU,cry,FUL,new,WHU,lee,MUN,BRE,tot,che,MCI,nfo,WOL,EVE,ars,SOU,avl
2,FUL,LIV,wol,BRE,ars,BHA,tot,CHE,nfo,NEW,whu,BOU,AVL,lee,EVE,mci,MUN,cry,SOU,lei,new,TOT,che,NFO,bha,WOL,bre,ARS,liv,bou,WHU,eve,LEE,avl,MCI,LEI,sou,CRY,mun


['BRE', 'BHA', 'FUL']

In [32]:
solve_N_pair_problem(N=2, first_gw=1, last_gw=19, max_iter=1)

NOTE: Initialized model N_rotation_pairs.

Selected: ARS and NEW. Total FDR: 1243.62, FDR per Fixture: 65.45


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,ARS,0.,71.310,58.930,64.110,76.220,74.670,63.690,-0.,0.,0.,63.040,0.,61.930,56.510,0.,69.580,0.,0.,71.710
1,NEW,56.510,0.,0.,0.,0.,0.,0.,58.930,64.110,72.400,0.,63.690,0.,0.,61.930,0.,71.310,63.040,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.,9.,10.,11.,12.,13.,14.,15.,16.,17.,18.,19.
0,ARS,cry,LEI,bou,FUL,AVL,mun,EVE,bre,TOT,LIV,lee,MCI,sou,NFO,che,wol,WHU,bha,NEW
1,NEW,NFO,bha,MCI,wol,liv,CRY,whu,BOU,ful,BRE,mun,EVE,tot,AVL,sou,CHE,lei,LEE,ars


In [33]:
solve_N_pair_problem(N=2, first_gw=1, last_gw=19, max_iter=1, exclude=['MCI', 'CHE', 'LIV', 'MUN', 'LEI', 'ARS', 'TOT'])

NOTE: Initialized model N_rotation_pairs.

Selected: BHA and FUL. Total FDR: 1249.25, FDR per Fixture: 65.75


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,BHA,74.670,0.,0.,63.040,64.110,71.310,58.930,0.,0.,0.,0.,56.510,0.,0.,69.580,0.,61.930,0.,63.690
1,FUL,0.,69.580,72.400,0.,0.,0.,0.,56.510,71.710,73.020,58.930,0.,63.040,63.690,0.,74.670,0.,61.930,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.,9.,10.,11.,12.,13.,14.,15.,16.,17.,18.,19.
0,BHA,mun,NEW,whu,LEE,ful,LEI,bou,CRY,liv,TOT,bre,NFO,mci,CHE,wol,AVL,sou,ARS,eve
1,FUL,LIV,wol,BRE,ars,BHA,tot,CHE,nfo,NEW,whu,BOU,AVL,lee,EVE,mci,MUN,cry,SOU,lei


In [34]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from itertools import repeat
gw_range = list(range(1,1))
start_gw = list(range(1,5))
all_pairs = [(sw, g) for sw in start_gw for g in gw_range]
# for gw_range in range(1, 10):
#     solve_N_pick_K_pair_problem(N=1, K=1, first_gw=1, last_gw=1+gw_range-1, max_iter=1, hfa=0.15)
with ThreadPoolExecutor(max_workers=16) as executor:
    res = list(executor.map(lambda x: solve_N_pick_K_pair_problem(**x), [{'N': 1, 'K': 1, 'first_gw': sw, 'last_gw': sw+g-1, 'max_iter': 1, 'hfa': 0.0} for sw in start_gw for g in gw_range]))

all_res = list(zip(all_pairs, res))
all_res

[]

In [35]:
solve_N_pick_K_pair_problem(N=2, K=1, max_iter=1, first_gw=1, last_gw=8, exclude=[], rating_type='def_rating')

NOTE: Initialized model N_K_rotation_problem_name.

Selected: ARS and WOL. Total FDR: -7.95, FDR per Fixture: -0.99


,team,1,2,3,4,5,6,7,8
0,ARS,0.,0.,1.,1.,0.,0.,0.990,0.790
1,WOL,1.070,1.,0.,0.,1.,1.100,0.,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,ARS,cry,LEI,bou,FUL,AVL,mun,EVE,bre
1,WOL,lee,FUL,tot,NEW,bou,SOU,liv,MCI


['ARS', 'WOL']

In [36]:
solve_N_pick_K_pair_problem(N=2, K=1, max_iter=1, first_gw=1, last_gw=8, exclude=['MCI', 'LIV', 'ARS', 'TOT', 'CHE'], rating_type='def_rating')

NOTE: Initialized model N_K_rotation_problem_name.

Selected: NFO and WOL. Total FDR: -7.94, FDR per Fixture: -0.99


,team,1,2,3,4,5,6,7,8
0,NFO,0.,0.,0.990,0.,0.,0.,1.070,1.
1,WOL,1.070,1.,0.,0.710,1.,1.100,0.,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,NFO,new,WHU,eve,TOT,mci,BOU,lee,FUL
1,WOL,lee,FUL,tot,NEW,bou,SOU,liv,MCI


['NFO', 'WOL']

In [37]:
solve_N_pick_K_pair_problem(N=2, K=1, max_iter=1, first_gw=1, last_gw=8, exclude=[], rating_type='off_rating')

NOTE: Initialized model N_K_rotation_problem_name.

Selected: MCI and NEW. Total FDR: 13.5, FDR per Fixture: 1.69


,team,1,2,3,4,5,6,7,8
0,MCI,0.,1.560,1.840,0.,1.450,0.,0.,0.
1,NEW,1.450,0.,0.,1.780,0.,1.880,1.980,1.560


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,MCI,whu,BOU,new,CRY,NFO,avl,TOT,wol
1,NEW,NFO,bha,MCI,wol,liv,CRY,whu,BOU


['MCI', 'NEW']

In [38]:
solve_N_pick_K_pair_problem(N=2, K=1, max_iter=1, first_gw=1, last_gw=8, exclude=['MCI', 'LIV', 'ARS', 'TOT', 'CHE'], rating_type='off_rating')

NOTE: Initialized model N_K_rotation_problem_name.

Selected: BHA and NEW. Total FDR: 13.86, FDR per Fixture: 1.73


,team,1,2,3,4,5,6,7,8
0,BHA,0.,1.840,1.980,0.,1.810,0.,1.560,0.
1,NEW,1.450,0.,0.,1.780,0.,1.880,0.,1.560


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,BHA,mun,NEW,whu,LEE,ful,LEI,bou,CRY
1,NEW,NFO,bha,MCI,wol,liv,CRY,whu,BOU


['BHA', 'NEW']

In [39]:
solve_N_pick_K_pair_problem(N=3, K=2, max_iter=1, first_gw=1, last_gw=8, exclude=[], rating_type='spi_rating')

NOTE: Initialized model N_K_rotation_problem_name.

Selected: ARS and NFO and WOL. Total FDR: 1047.86, FDR per Fixture: 130.98


,team,1,2,3,4,5,6,7,8
0,ARS,0.,71.310,58.930,64.110,76.220,0.,63.690,72.400
1,NFO,71.710,0.,63.690,0.,0.,58.930,63.040,64.110
2,WOL,63.040,64.110,0.,71.710,58.930,61.930,0.,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,ARS,cry,LEI,bou,FUL,AVL,mun,EVE,bre
1,NFO,new,WHU,eve,TOT,mci,BOU,lee,FUL
2,WOL,lee,FUL,tot,NEW,bou,SOU,liv,MCI


['ARS', 'NFO', 'WOL']

In [40]:
solve_N_pick_K_pair_problem(N=3, K=2, max_iter=1, first_gw=1, last_gw=8, exclude=['MCI', 'LIV', 'ARS', 'TOT', 'CHE'], rating_type='spi_rating')

NOTE: Initialized model N_K_rotation_problem_name.

Selected: BHA and NFO and WOL. Total FDR: 1048.49, FDR per Fixture: 131.06


,team,1,2,3,4,5,6,7,8
0,BHA,0.,71.710,73.020,63.040,64.110,0.,58.930,76.480
1,NFO,71.710,0.,63.690,0.,0.,58.930,63.040,64.110
2,WOL,63.040,64.110,0.,71.710,58.930,61.930,0.,0.


,team,1.,2.,3.,4.,5.,6.,7.,8.
0,BHA,mun,NEW,whu,LEE,ful,LEI,bou,CRY
1,NFO,new,WHU,eve,TOT,mci,BOU,lee,FUL
2,WOL,lee,FUL,tot,NEW,bou,SOU,liv,MCI


['BHA', 'NFO', 'WOL']